In [1]:
# import easydict
from multiprocessing import Process
import yaml
from pathlib import Path
import argparse
import torch
import tqdm
import numpy as np

from yolov5.train_dt import yolov5
from EfficientObjectDetection.train_new_reward import EfficientOD
from utils import load_filenames, load_dataset, load_dataloader, compute_map

In [2]:
import munch

In [3]:
opt = {'epochs':100,
      'batch_size':24,
      'device':2,
      'test_epoch':10,
      'eval_epoch':2,
      'step_batch_size':100,
      'save_path':'save',
      'rl_weight':None,
      'h_detector_weight':'',
      'l_detector_weight':'',
      'fine_tr':'config/fine_tr.yaml',
      'fine_eval':'config/fine_eval.yaml',
      'coarse_tr':'config/coarse_tr.yaml',
      'coarse_eval':'config/coarse_eval.yaml',
      'EfficientOD':'config/EfficientOD.yaml'}

opt = munch.AutoMunch(opt)

In [4]:
# training option load from yaml files
with open(opt.fine_tr) as f:
    fine_tr = yaml.load(f, Loader=yaml.FullLoader)
with open(opt.fine_eval) as f:
    fine_eval = yaml.load(f, Loader=yaml.FullLoader)
with open(opt.coarse_tr) as f:
    coarse_tr = yaml.load(f, Loader=yaml.FullLoader)
with open(opt.coarse_eval) as f:
    coarse_eval = yaml.load(f, Loader=yaml.FullLoader)
with open(opt.EfficientOD) as f:
    efficient_config = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
efficient_config['load'] = None

In [6]:
epochs = opt.epochs
bs = opt.batch_size
fine_detector = yolov5(fine_tr, fine_eval, epochs, bs)
coarse_detector = yolov5(coarse_tr, coarse_eval, epochs, bs)
rl_agent = EfficientOD(efficient_config)


                 from  n    params  module                                  arguments                     
  0                -1  1      8800  yolov5.models.common.Focus              [3, 80, 3]                    
  1                -1  1    115520  yolov5.models.common.Conv               [80, 160, 3, 2]               
  2                -1  1    315680  yolov5.models.common.BottleneckCSP      [160, 160, 4]                 
  3                -1  1    461440  yolov5.models.common.Conv               [160, 320, 3, 2]              
  4                -1  1   3311680  yolov5.models.common.BottleneckCSP      [320, 320, 12]                
  5                -1  1   1844480  yolov5.models.common.Conv               [320, 640, 3, 2]              
  6                -1  1  13228160  yolov5.models.common.BottleneckCSP      [640, 640, 12]                
  7                -1  1   7375360  yolov5.models.common.Conv               [640, 1280, 3, 2]             
  8                -1  1   4099840  

In [7]:
split_train_path = '/home/SSDD/ICIP21_dataset/split_data_4_0/rl_ver/test/images'
split_val_path = '/home/SSDD/ICIP21_dataset/split_data_4_0/rl_ver/test/images'
split_test_path = '/home/SSDD/ICIP21_dataset/split_data_4_0/rl_ver/test/images'
split = 4

original_img_path = '/home/SSDD/ICIP21_dataset/origin_data/rl_ver/test/images'

assert bs % split == 0, 'batch size should be divided with image split patch size'


In [8]:
for e in range(epochs):
    train_imgs = load_filenames(split_train_path, split, bs).files_array()
    fine_train_dataset = load_dataset(train_imgs, fine_tr, bs)
    coarse_train_dataset = load_dataset(train_imgs, fine_tr, bs)

    fine_train_loader = load_dataloader(bs, fine_train_dataset)
    coarse_train_loader = load_dataloader(bs, coarse_train_dataset)

    fine_train_nb = len(fine_train_loader)
    coarse_train_nb = len(coarse_train_loader)
    assert fine_train_nb == coarse_train_nb, 'fine & coarse train batch number is not matched'
    nb = fine_train_nb

    for i, (fine_train, coarse_train) in tqdm.tqdm(enumerate(zip(fine_train_loader, coarse_train_loader)),
                                                   total=fine_train_nb):
        fine_detector.train(e, i, nb, fine_train_dataset, fine_train)
        coarse_detector.train(e, i, nb, coarse_train_dataset, coarse_train)

        # result = (source_path, paths[si], mp, mr, map50, nl, stats)
        fine_results = fine_detector.eval(fine_train)
        coarse_results = coarse_detector.eval(coarse_train)

        rl_agent.train(e, i, nb, fine_results, coarse_results)
        
        break
        
    # Validation
    if e % 10 == 0:
        fine_dataset, coarse_dataset, policies = rl_agent.eval(split_val_path, original_img_path)

        fine_val_dataset = load_dataset(fine_dataset, fine_tr, bs)
        coarse_val_dataset = load_dataset(coarse_dataset, fine_tr, bs)

        fine_val_loader = load_dataloader(bs, fine_val_dataset)
        coarse_val_loader = load_dataloader(bs, coarse_val_dataset)

        fine_nb = len(fine_val_loader)
        coarse_nb = len(coarse_train_loader)

        fine_results, coarse_results = [], []
        for i, fine_val in tqdm.tqdm(enumerate(fine_val_loader), total=fine_nb):
            fine_results.append(fine_detector.eval(fine_val))

        for i, coarse_val in tqdm.tqdm(enumerate(coarse_val_loader), total=coarse_nb):
            coarse_results.append(coarse_detector.eval(coarse_val))

        map50 = compute_map(fine_results, coarse_results)
        print('MAP: \n', map50 )



Scanning images: 100%|██████████| 7848/7848 [00:05<00:00, 1366.66it/s]
Scanning labels /home/SSDD/ICIP21_dataset/split_data_4_0/rl_ver/test/yolov5_txt.cache (3556 found, 0 missing, 4292 empty, 0 duplicate, for 7848 images): 100%|██████████| 7848/7848 [00:00<00:00, 32546.81it/s]
Scanning labels /home/SSDD/ICIP21_dataset/split_data_4_0/rl_ver/test/yolov5_txt.cache (3556 found, 0 missing, 4292 empty, 0 duplicate, for 7848 images): 100%|██████████| 7848/7848 [00:00<00:00, 33091.16it/s]
  0%|          | 0/327 [00:00<?, ?it/s]

Image sizes 480 train, 480 test
Starting training for 0 epochs...
Image sizes 96 train, 96 test
Starting training for 0 epochs...
(0,)
(0,)
(1,)
(1,)
(1,)
(0,)
(0,)
(0,)
(1,)
(1,)
(1,)
(2,)
(0,)
(0,)
(1,)
(0,)
(0,)
(2,)
(0,)
(5,)
(1,)
(0,)
(0,)
(0,)


  0%|          | 0/327 [00:11<?, ?it/s]

(0,)
(0,)
(1,)
(1,)
(1,)
(0,)
(0,)
(0,)
(1,)
(1,)
(1,)
(2,)
(0,)
(0,)
(1,)
(0,)
(0,)
(2,)
(0,)
(5,)
(1,)
(0,)
(0,)
(0,)



Scanning images: 100%|██████████| 4706/4706 [00:03<00:00, 1413.68it/s]
Scanning labels /home/SSDD/ICIP21_dataset/split_data_4_0/rl_ver/test/yolov5_txt.cache (2122 found, 0 missing, 2584 empty, 0 duplicate, for 4706 images): 100%|██████████| 4706/4706 [00:00<00:00, 33813.21it/s]
Scanning images: 100%|██████████| 3142/3142 [00:02<00:00, 1410.83it/s]
Scanning labels /home/SSDD/ICIP21_dataset/split_data_4_0/rl_ver/test/yolov5_txt.cache (1434 found, 0 missing, 1708 empty, 0 duplicate, for 3142 images): 100%|██████████| 3142/3142 [00:00<00:00, 33223.68it/s]
  1%|          | 1/197 [00:00<02:56,  1.11it/s]

...

 40%|███▉      | 130/327 [00:55<01:22,  2.39it/s]

(0,)
(0,)
(1,)
(0,)
(1,)
(0,)
(1,)
(1,)
(0,)
(0,)
(0,)
(0,)
(1,)
(0,)
(0,)
(0,)
(1,)
(0,)
(1,)
(0,)
(0,)
(0,)
(0,)
(0,)


 40%|████      | 131/327 [00:57<01:26,  2.27it/s]

(0,)
(0,)
(1,)
(1,)
(0,)
(0,)
(0,)
(0,)
(0,)
(1,)
(0,)
(1,)
(0,)
(0,)
(0,)
(1,)
(0,)
(0,)
(0,)
(0,)
(0,)
(0,)


IndexError: list index out of range

In [ ]:
# inf_out: (1, 14175, 6)
# train_out: list ( ,6)

In [ ]:
# dataset length 4 (img, target, path, info)
fine_train[0].shape

In [ ]:
fine_train[1]

In [ ]:
fine_train[3]

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)

# # replace the classifier with a new one, that has
# # num_classes which is user-defined
num_classes = 1  # 1 class (person) + background
# # get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# # replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [ ]:
model.forward(fine_train[0], fine_train[1][:,:])

In [ ]:
def yolo2coco(tensor, org_res):
    # cx, cy, w, y --> x0, y0, y1, y2
    w, h = tensor[2]/2, tensor[3]/2
    x0, x1 = int((tensor[0]-w)*org_res), int((tensor[0]+w)*org_res)
    y0, y1 = int((tensor[1]-h)*org_res), int((tensor[1]+h)*org_res)
    return torch.Tensor([x0, y0, x1, y1]).unsqueeze(0)

def convert_yolo2coco(targets, org_res=479):
    # (img_id, cls, bb4)
    res = []
    for target in targets:
        dummy_dict = {}
        dummy_dict['image_id'] = torch.Tensor((target[0],)).type(torch.int64).cuda()
        dummy_dict['labels'] = torch.Tensor((target[1],)).type(torch.int64).cuda()
        dummy_dict['boxes'] = yolo2coco(target[2:], org_res).cuda()
        res.append(dummy_dict)
    return res
    

In [ ]:
new_t = convert_yolo2coco(fine_train[1])

In [ ]:
model.cuda()

In [ ]:
model((fine_train[0]/255.).cuda()[:5], new_t[:5])

In [ ]:
new_t[0]['image_id']

In [ ]:
(fine_train[0]/255.)[:5].shape

In [ ]:
len(new_t[:5])